In [1]:
from besluit import besluit_plaintext, besluit_plaintext_2

In [2]:
import ollama

from ollama import Client

client = Client(host='http://ollama:11434')


def response(prompt):


    client.pull('llama3.1')
    response = client.generate(model='llama3.1', prompt=prompt)
    # response = client.chat("llama3.1", messages=[
    # {
    #     'role': 'user',
    #     'content': 'Why is the sky blue?',
    # },    ])
    return response


# Summarize the Besluit

In [43]:
from prompts import summary_prompt

In [5]:
prompt = """
You are a document summarizer. Based on the regulation document you received, you create a clear summary of the regulation. 
The summary must contain between 200 and 500 words.

Answer in **Dutch**

Mention clearly what is the Product that is describe in the reglementation.

document: 
"""

prompt +=besluit_plaintext

In [6]:
res = response(prompt)
print(res['response'])
summary = res['response']

**Samenvatting van het reglement**

Het reglement "Retributiereglement voor het gebruik van repetitielokaal The Basement Lievegem" heeft tot doel retributies vast te leggen die zijn verschuldigd door de gebruikers van de repetitieruimte in Lievegem.

**Product**: Het product dat wordt beschreven in dit reglement is de repetitieruimte "The Basement" in 't Kwadrant, Hofbouwstraat 1, 9930 Lievegem.

Het reglement bevat drie artikelen:

* Artikel 2 definieert de tarieven voor het gebruik van de repetitieruimte: €2 per uur voor individuele beoefenaars (maximaal 4 uur aaneensluitend), €4 per uur voor groepen (maximaal 4 uur aaneensluitend), €19 per dag en €190 per jaar.
* Artikel 3 beschrijft de betalingsmodaliteiten: de retributie moet worden betaald binnen 30 dagen na verzending van de factuur, en bij gebrek aan betaling zal de invordering van onbetwiste en opeisbare niet-fiscale ontvangsten gebeuren.
* Artikel 4 beschrijft het oneigenlijk gebruik van de ruimte: de gebruiker verklaart dat 

# Extract the associated product

In [7]:
from prompts import product_prompt

In [21]:
res_product = response(f"{product_prompt}\n{summary}")
product = res_product['response']  
print(product)
product = product.split("<Product>")[-1].split("<\Product>")[0]

<Product> Retributiereglement voor het gebruik van repetitielokaal The Basement Lievegem <\Product>


In [22]:
product

' Retributiereglement voor het gebruik van repetitielokaal The Basement Lievegem '

Describe the product in details

In [10]:
from prompts import prompt_description
pr = f"{prompt_description}\nProduct name : {product}\nregulation{besluit_plaintext}"

In [11]:
res_description = response(pr)
product_description = res_product['response']  
print(product_description)

<Product> Retributiereglement voor het gebruik van repetitielokaal The Basement Lievegem <\Product>

Dit reglement bevat een beschrijving van de tarieven, betalingsmodaliteiten en oneigenlijk gebruik van de repetitieruimte "The Basement" in 't Kwadrant, Hofbouwstraat 1, 9930 Lievegem.


In [18]:
from prompts import prompt_multi_fields


In [19]:
res_multifiel = response(prompt_multi_fields.format(product_name=product, regulation=besluit_plaintext))
print(res_multifiel["response"])

Here are the extracted data:

**Conditions associated to the product:**

<Condition>Uitsluitend voor individuele beoefenaars en groepen.</Condition>
<Condition>Tarief omvat naast het gebruik van de zaal ook de aanwezige basisuitrusting en energiekosten.</Condition>

**Proof that needs to be provided:**

<Proof>Betalingsbewijs of bewijs dat de factuur is betaald.</Proof>
<Proof>Factuur met een betalingstijd van 30 dagen na verzending.</Proof>

**Procedure to access the product:**

<Procedure>Gebruikers dienen zich te registreren en een aanvraagformulier in te dienen om de repetitieruimte te reserveren.</Procedure>
<Procedure>Repetitielokaal wordt opgelost bij gebrek aan betaling van factuur, behoudens het recht om in te spreken binnen 30 dagen na verzending.</Procedure>

Let me know if you'd like me to clarify anything!


In [26]:
r = response("""transform this LLM output in a json with fields like this one
             {
              Conditions :[],
              Procedure :[],
              Proof :[] 
             }
             
             \n"""+res_multifiel["response"])

In [27]:
print(r['response'])

Here is the transformed LLM output into a JSON format with the specified fields:

```
{
  "Conditions": [
    "Uitsluitend voor individuele beoefenaars en groepen.",
    "Tarief omvat naast het gebruik van de zaal ook de aanwezige basisuitrusting en energiekosten."
  ],
  "Procedure": [
    "Gebruikers dienen zich te registreren en een aanvraagformulier in te dienen om de repetitieruimte te reserveren.",
    "Repetitielokaal wordt opgelost bij gebrek aan betaling van factuur, behoudens het recht om in te spreken binnen 30 dagen na verzending."
  ],
  "Proof": [
    "Betalingsbewijs of bewijs dat de factuur is betaald.",
    "Factuur met een betalingstijd van 30 dagen na verzending."
  ]
}
```

Let me know if you'd like me to make any adjustments!


# Besluit Title 
This is a prompt to get title from the Besluit. 

In [18]:
prompt_title = """
You give title to a product or service that are derived from reglementations.
Below you will receive a summary of the documentation. Extract the title from the associated product. 

The titles are in Dutch and contains 30 words. The title are between the separator <TITLE>
Answer in Dutch

Here are three example : 
document 1 : "
Hier is een samenvatting van het document in maximaal 500 woorden:

**Retributiereglement voor het gebruik van repetitielokaal The Basement Lievegem**

Het retributiereglement regelt de rechten en plichten die verbonden zijn aan het gebruik van de repetitieruimte "The Basement" in Lievegem. Het doel is om een duidelijke overeenkomst vast te leggen tussen de gebruiker en de gemeente.

**Tarieven**

* Individuele beoefenaars: €2 per uur (maximaal 4 uur aaneensluitend)
* Groepen: €4 per uur (maximaal 4 uur aaneensluitend)
* Dagtarief: €19 (na afspraak met Team Evenementen)
* Jaartarief: €190 per jaar voor groepen en individuele beoefenaars

Deze tarieven omvatten de kosten van het gebruik van de zaal, basisuitrusting en energiekosten.

**Betalingsmodaliteiten**

* De retributie moet worden betaald binnen 30 dagen na verzending van de factuur.
* Bij gebrek aan betaling zal de invordering van onbetwiste en opeisbare niet-fiscale ontvangsten gebeuren.
* Betwistingen van de factuur moeten worden gedaan binnen 30 dagen volgend op de verzending van de factuur.

**Oneigenlijk gebruik van de ruimte**

* De gebruiker verklaart dat de gegevens op het aanvraagformulier conform zijn met betrekking tot de aanvrager en de geplande activiteit.
* De gebruiker kan de accommodatie niet ter beschikking stellen van derden.
* Indien zou blijken dat de werkelijke activiteit niet overeenstemt met de toegelaten activiteit wordt een schadevergoeding opgelegd van €125.

**Modaliteiten**

* Alle modaliteiten rond het gebruik van de repetitieruimte worden beschreven in het gebruikersreglement.

Het retributiereglement is vastgesteld door de gemeenteraad en treedt in werking vanaf 1 juni 2024. Het oude retributiereglement wordt opgeheven.
"
Product Title 1 : "Het gebruik van repetitielokaal The Basement Lievegem"

Predict the title for the product associated to the  : 
"""

prompt_title +=summary

In [19]:
res_title = response(prompt_title)
title = res_title['response']  
print(title)

Ik kan de titel extraheren uit het beschreven document:

<TITLE>Reglement voor de toekenning van een stedelijke premie in Turnhout</TITLE>

Dus de producttitle is: "Reglement voor de toekenning van een stedelijke premie in Turnhout"
